# _Notebook Jupyter_ 6_NLP_modeloClassificador_parteManual

# Classificação da Aplicação divergente de forma manual

## Importando bibliotecas

In [1]:
import pandas as pd, numpy as np, re, time

In [2]:
# Data e hora da execução do script
print(f'Código executado em {time.strftime("%d/%m/%Y às %H:%M", time.localtime(time.time()))}')

Código executado em 12/01/2022 às 19:38


## Carregando dataset

In [3]:
# Importa base de dados com os modelos já determinados para um dataframe
df = pd.read_excel(r'./bases/dataframe_modelos_classificado_manual.xlsx')
df.iloc[:,-5:].head()

,Modelo,APLICACAO,APLICACAOSVC,APLICACAOMNB,APLICACAOFIM
0,TITAN HONDA FAN CG TIT 150 AN,HONDA CG TIT TITAN 125 150 160,HONDA CG TIT TITAN 125 150 160,HONDA CG TIT TITAN 125 150 160,HONDA CG TIT TITAN 125 150 160
1,TITAN HONDA CG CARGO TIT 125 AN,HONDA CG TIT TITAN 125 150 160,HONDA CG TIT TITAN 125 150 160,HONDA CG TIT TITAN 125 150 160,HONDA CG TIT TITAN 125 150 160
2,125 CG HONDA FAN,HONDA CG FAN,HONDA CG FAN,HONDA CG FAN,HONDA CG FAN
3,HONDA BIZ C100,HONDA BIZ 100 C100 125 C125,HONDA BIZ 100 C100 125 C125,HONDA BIZ 100 C100 125 C125,HONDA BIZ 100 C100 125 C125
4,MIRAGE 150,KASINSKI MIRAGE 150 250,KASINSKI MIRAGE 150 250,KASINSKI MIRAGE 150 250,KASINSKI MIRAGE 150 250


In [4]:
# Verifica o tamnanho do dataframe
df.shape

(18327, 9)

### Determinando os registros divergentes

In [5]:
# Registros ainda não definidos (lista de opções)
# O filtro definirá True se APLICACAOFIM iniciar com '[' ou False caso contrário.
filtro=[]
for i, aplicacao in enumerate(df['APLICACAOFIM']):
    if aplicacao[0]=='[':
        filtro.append(True)
    else:
        filtro.append(False)

In [6]:
df[filtro].iloc[:,-5:]

,Modelo,APLICACAO,APLICACAOSVC,APLICACAOMNB,APLICACAOFIM
11,125 HUNTER MAX,XXX,SUNDOWN HUNTER,HONDA CT 125 HUNTER,"['SUNDOWN HUNTER', 'HONDA CT 125 HUNTER']"
16,YAMAHA TENERE XT 250,XXX,YAMAHA XTZ TENERE 250,YAMAHA XT TENERE 660Z 660,"['YAMAHA XTZ TENERE 250', 'YAMAHA XT TENERE 66..."
289,125 HUNTER MAX,XXX,SUNDOWN HUNTER,HONDA CT 125 HUNTER,"['SUNDOWN HUNTER', 'HONDA CT 125 HUNTER']"
311,125 HUNTER MAX,XXX,SUNDOWN HUNTER,HONDA CT 125 HUNTER,"['SUNDOWN HUNTER', 'HONDA CT 125 HUNTER']"
632,HONDA CB MAX 300,XXX,SUNDOWN MAX,HONDA CB 300R 300 CB300,"['SUNDOWN MAX', 'HONDA CB 300R 300 CB300']"
873,125 HUNTER MAX,XXX,SUNDOWN HUNTER,HONDA CT 125 HUNTER,"['SUNDOWN HUNTER', 'HONDA CT 125 HUNTER']"
964,HONDA XR NX 200,XXX,HONDA XR,HONDA NX 150 200 250,"['HONDA XR', 'HONDA NX 150 200 250']"
1093,YAMAHA TENERE XT 250,XXX,YAMAHA XTZ TENERE 250,YAMAHA XT TENERE 660Z 660,"['YAMAHA XTZ TENERE 250', 'YAMAHA XT TENERE 66..."
1126,YAMAHA TENERE XT 250,XXX,YAMAHA XTZ TENERE 250,YAMAHA XT TENERE 660Z 660,"['YAMAHA XTZ TENERE 250', 'YAMAHA XT TENERE 66..."
1411,125 HUNTER MAX,XXX,SUNDOWN HUNTER,HONDA CT 125 HUNTER,"['SUNDOWN HUNTER', 'HONDA CT 125 HUNTER']"


In [7]:
df[filtro].shape

(149, 9)

### Classificando os registros divergentes

In [8]:
def ordena(modelo):
    modelo=modelo.split()
    modelo.sort()
    modelo=" ".join(modelo)
    return modelo
def limpaColchetes(aplicacaofim):
    aplicacaofim=" ".join(aplicacaofim)
    aplicacaofim=aplicacaofim.replace("'",'')
    aplicacaofim=aplicacaofim.replace("[",'')
    aplicacaofim=aplicacaofim.replace("]",'')
    aplicacaofim=aplicacaofim.split(",")
    return aplicacaofim

In [9]:
lista=df[filtro].groupby('Modelo').agg(lambda x: list(set(x))).reset_index()[['Modelo','APLICACAOFIM']]
lista['Modelo']=lista['Modelo'].apply(ordena)
lista['APLICACAOFIM']=lista['APLICACAOFIM'].apply(limpaColchetes)
lista.shape

(25, 2)

Obsrva-se que são pouquíssimos casos a analisar.

In [10]:
lista.sample(3)

,Modelo,APLICACAOFIM
7,110 300 CB HONDA,"[HONDA CRF 110 150 110/150, HONDA CB 300R 300..."
3,125 HONDA WR XL,"[HUSQVARNA WR, HONDA XL XLS 125 XL125 XL125S]"
9,300 CB HONDA MAX,"[SUNDOWN MAX, HONDA CB 300R 300 CB300]"


In [11]:
# com essa lista bastará deixar a opção correta.
relacoes={}
for i, x in enumerate(lista['Modelo']):
    relacoes[x]=lista['APLICACAOFIM'][i]

In [12]:
relacoes

{'100 MOTO': ['APRILIA MOTO', ' KAWASAKI KX 100 125 100/125'],
 '125 HUNTER MAX': ['SUNDOWN HUNTER', ' HONDA CT 125 HUNTER'],
 '125 HONDA RACING XL': ['SHINERAY RACING 200',
  ' HONDA XL XLS 125 XL125 XL125S'],
 '125 HONDA WR XL': ['HUSQVARNA WR', ' HONDA XL XLS 125 XL125 XL125S'],
 '110 250 CBX HONDA': ['HONDA CBX 1000', ' HONDA TWISTER CBX 250'],
 '110 BIZ HONDA MOTO': ['HONDA BIZ 100 C100 125 C125',
  ' APRILIA MOTO',
  ' HONDA CRF 110 150 110/150'],
 '110 250 CB HONDA': ['HONDA CRF 110 150 110/150', ' HONDA CB 250 250F'],
 '110 300 CB HONDA': ['HONDA CRF 110 150 110/150', ' HONDA CB 300R 300 CB300'],
 '500 CB GOLD HONDA': ['MVK FENIX GOLD', ' HONDA CB 500'],
 '300 CB HONDA MAX': ['SUNDOWN MAX', ' HONDA CB 300R 300 CB300'],
 '250F CB CITY ERO HONDA TWISTER': ['HONDA TWISTER CBX 250',
  ' KASINSKI ERO',
  ' HONDA CB 250 250F'],
 '110 230 CRF HONDA': ['HONDA CRF 230 230F 250 250F',
  ' HONDA CRF 110 150 110/150'],
 '200 CBX HONDA WR': ['HUSQVARNA WR', ' HONDA STRADA CBX 200'],
 '200 H

In [13]:
# Escolhas das relações realizada manualmente (somente 26 - deixamos uma de fora de propoósito.)
relacoes={'110 50 SHINERAY': 'SHINERAY NEW 50',
          '125 HUNTER MAX': 'SUNDOWN HUNTER',
          '110 250 CBX HONDA': 'HONDA TWISTER CBX 250',
          '110 230 CRF HONDA': 'HONDA CRF 230 230F 250 250F',
          '125 HONDA RACING XL': 'HONDA XL XLS 125 XL125 XL125S',
          '125 HONDA WR XL': 'HONDA XL XLS 125 XL125 XL125S',
          '500 CB GOLD HONDA': 'HONDA CB 500',
          '110 BIZ EVO HONDA': 'HONDA BIZ 100 C100 125 C125',
          '110 250 CB HONDA': 'HONDA CB 250 250F',
          '110 300 CB HONDA': 'HONDA CB 300R 300 CB300',
          '300 CB HONDA MAX': 'HONDA CB 300R 300 CB300',
          '600R CBR GOLD HONDA R60': 'HONDA CBR 250',
          '200 HONDA NX XR': 'HONDA NX 150 200 250',
          '250F CB CITY ERO HONDA TWISTER': 'HONDA TWISTER CBX 250',
          '200 CBX HONDA WR': 'HONDA STRADA CBX 200',
          '50 FOX JET SHINERAY SMART': 'SHINERAY JET 50',
          '150 BLACK FAZER YAMAHA': 'YAMAHA FAZER YS150 150',
          '150 FAZER GT YAMAHA': 'YAMAHA FAZER YS150 150',
          '250 FAZER GT YAMAHA': 'YAMAHA FAZER YS250 250',
          '150 FAZER MAX YAMAHA': 'YAMAHA FAZER YS150 150',
          '150 FAZER MOTO YAMAHA': 'YAMAHA FAZER YS150 150',
          '150 FAZER RACING YAMAHA': 'YAMAHA FAZER YS150 150',
          '250 TENERE XT YAMAHA': 'YAMAHA XTZ TENERE 250',
          '150 FAZER WR YAMAHA': 'YAMAHA FAZER YS150 150',
          '150 MAX XTZ YAMAHA': 'YAMAHA XTZ 125',
          '110 BIZ HONDA MOTO': 'HONDA BIZ 100 C100 125 C125',
          '160 BROS HONDA WR': 'HONDA NXR 150 BROS BROZ'}

In [14]:
colindex = df.columns.get_loc("APLICACAOFIM")
for i, row in df.iterrows():
    if row['APLICACAOFIM'][0]=='[':
        try:
            modelo=row['Modelo'].split()
            modelo.sort()
            modelo=" ".join(modelo)
            relacoes[modelo]
        except:
            continue
        df.iloc[i,colindex]=relacoes[modelo]

Reaplicando o filtro

In [15]:
# Registros ainda não definidos (lista de opções)
# O filtro definirá True se APLICACAOFIM iniciar com '[' ou False caso contrário.
filtro=[]
for i, aplicacao in enumerate(df['APLICACAOFIM']):
    if aplicacao[0]=='[':
        filtro.append(True)
    else:
        filtro.append(False)

In [16]:
df[filtro].shape

(1, 9)

Precisamos agora fazer a observação manual dos registros divergentes para correção.<br>
A seguir, para os registros que não estiverem no diconário, cada linha apresentará a escolha manual de qual das opções deverá ser a APLICACAOFIM.

In [17]:
# As escolhas serão em um dicionário para poder retomar
# {index: escolha} ==> exemplo: {11: 1, 16: 1}
escolhas={}
from pathlib import Path
fileName = r"escolhas.npy"
fileObj = Path(fileName)
if fileObj.is_file():
    escolhas=np.load('escolhas.npy')
    escolhas=dict(enumerate(escolhas.flatten(), 1))[1]
else:
    escolhas={}

In [18]:
len(escolhas.keys())

0

In [19]:
colindex = df[filtro].columns.get_loc("APLICACAOFIM")
for i, row in df[filtro].iterrows():
    print(i)
    ultimo_i=max(escolhas.keys()) if len(escolhas.keys())>0 else 0
    a=0
    if i>100000 or a=='X': # serve para cancelar e estabelecer um limite para as verificações até o índice i
        break
    if row['APLICACAOFIM'][0]=='[':
        # guarda a lista das opções na variável
        aplictemp=row['APLICACAOFIM'].replace("[","").replace(", ",";").replace(",",";").replace("'","").replace("]","")
        aplictemp=aplictemp.split(';')
        print('*'*20)
        print('Digite o número de uma das opções e tecle Enter:')
        print('Modelo: ', row['Modelo'])
        for n, aplicacao in enumerate(aplictemp):
            aplictemp[n]=aplicacao.strip()
            print(' ',str(n+1)+')',aplictemp[n])
        pedeEscolha=False
        if ultimo_i>=i:
            try:
                a=escolhas[i]
            except:
                pedeEscolha=True
        if pedeEscolha or ultimo_i<i:
            while True:
                try:
                    a=input('Escolha uma das opções acima ou X para cancelar: ')
                    if a=='X':
                        print(a)
                        break
                    a=int(a)
                    if a not in range(1,len(aplictemp)+1):
                        raise(ValueError)
                except ValueError:
                    print("\nOpção inválida.\nDigite o número de uma das opções apresentadas:")
                    #print('Modelo: ', row['Modelo'])
                    #for n, aplicacao in enumerate(aplictemp):
                        #print(' ',str(n+1)+')',aplictemp[n])
                else:
                    break
            if a=='X':
                break
        print(aplictemp[a-1],'\n\n')
        df.iloc[i,colindex]=aplictemp[a-1]
        escolhas[i]=a # acrescenta ou atualiza a escolha no dicionário
    np.save('escolhas.npy', escolhas) # salva o dicionário em um arquivo
print(escolhas)

14437
********************
Digite o número de uma das opções e tecle Enter:
Modelo:  100 MOTO
  1) APRILIA MOTO
  2) KAWASAKI KX 100 125 100/125
Escolha uma das opções acima ou X para cancelar: 2
KAWASAKI KX 100 125 100/125 


{14437: 2}


Depois da análise manual todos os registros estão classificados para podermos utilizar como base para treinar o nosso classificador definitivo.

In [20]:
# Registros ainda não definidos (lista de opções)
# O filtro definirá True se APLICACAOFIM iniciar com '[' ou False caso contrário.
filtro=[]
for i, aplicacao in enumerate(df['APLICACAOFIM']):
    if aplicacao[0]=='[':
        filtro.append(True)
    else:
        filtro.append(False)

In [21]:
df[filtro]

,PAIS DE ORIGEM,DESCRICAO DO PRODUTO,VALOR UN.PROD.DOLAR,DESCRICAO,Modelo,APLICACAO,APLICACAOSVC,APLICACAOMNB,APLICACAOFIM


In [22]:
df[filtro].shape

(0, 9)

In [23]:
df=df.assign(APLICACAO=df.APLICACAOFIM.tolist())

In [24]:
df = df.drop('APLICACAOSVC', 1)
df = df.drop('APLICACAOMNB', 1)
df = df.drop('APLICACAOFIM', 1)

In [25]:
df.iloc[0:,-3:].sample(5)

,DESCRICAO,Modelo,APLICACAO
5922,biz 125 nr,HONDA 125 BIZ,HONDA BIZ 100 C100 125 C125
9065,titan160 19 15 allen,TITAN 160 TIT,HONDA CG TIT TITAN 125 150 160
7400,darom xr 250 tornado,HONDA TORNADO 250 XR,HONDA TORNADO XR 250
17906,biz 110 14,HONDA BIZ 110,HONDA BIZ 100 C100 125 C125
15244,nxr 160 bros xre 428,HONDA BROS 160 XRE NXR,HONDA NXR 150 BROS BROZ


## Criando a coluna de existência de corrente com Retentor no Kit
Diante da possibilidade do kit de trasmissão vir acompanhado ou não de corrente com retentor, e esta questão influenciar no preço do produto, faz-se necessário criar uma coluna do tipo _boolean_ para indicar ou não a presença de corrente com retentor no kit.

Após a análise do dataset, observou-se que todos os kits que possuíam corrente com retentor havia na descrição uma das seguintes opções:<br>
* com retentor
* c/retentor
* c/ retentor
* com ret
* c/ret
* c/ ret

Desse modo, definiu-se o padrão Regex para encontrar essas formas na descrição e colocar True|False na coluna RETENTOR

### Função que determina a existência de retentor na corrente do kit e retorna True|False

In [26]:
def retentorAux(descricao):
    # define o padrão de busca
    padrao = r'c/ *ret|com *ret' #r"(com|c/) *ret"
    descricao=descricao.lower()
    busca = re.findall(padrao, descricao)
    if busca:
        descricao = busca[0]
        return True
    else:
        descricao=''
        return False

In [27]:
# cria a coluna RETENTOR com a indicação True|False
df['RETENTOR']=df['DESCRICAO DO PRODUTO'].apply(retentorAux)

## Exportando o DataSet Classificado

Exportando para um arquivo CSV

In [28]:
df.to_csv(r'./bases/dataframe_modelos_classificado.csv', index = False, header = True)

Exportando para um arquivo de planilha do Excel

In [29]:
df.to_excel(r'./bases/dataframe_modelos_classificado.xlsx', index = False, header = True)